<a href="https://colab.research.google.com/github/naraokarei/rcnn_Lung/blob/master/pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import gc
import sys
import time
import json
import glob
import random
from pathlib import Path
import pandas as pd

from PIL import Image
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa

import itertools
from tqdm import tqdm

In [ ]:
!pip install tensorflow==1.14.0
!pip install keras==2.2.5
import tensorflow as tf

     |████████████████████████████████| 109.2MB 38kB/s 
     |████████████████████████████████| 491kB 33.8MB/s 
     |████████████████████████████████| 3.2MB 41.5MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 337kB 2.9MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
! pip install pydicom
import pydicom

     |████████████████████████████████| 35.5MB 89kB/s 


In [ ]:
train_df = pd.read_csv("drive/My Drive/CLNDAT_EN.csv")
train_df.head()
train＿df["Sex"][train_df["Sex"] == "female"] = 1
train＿df["Sex"][train_df["Sex"] == "male"] = 0
train＿df["Result"][train_df["Result"] == "malignant"] = 1
train＿df["Result"][train_df["Result"] == "benign"] = 0
train_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

,ImageId,Sex,x,y,Result
0,JPCLN001,0,1634,692,1
1,JPCLN002,1,1614,1090,0
2,JPCLN003,1,1303,447,1
3,JPCLN004,1,606,836,0
4,JPCLN005,1,1438,692,1
...,...,...,...,...,...
149,JPCLN150,1,1544,1409,1
150,JPCLN151,1,1520,1364,1
151,JPCLN152,0,1323,1438,0
152,JPCLN153,0,1708,1122,1


In [ ]:
dicom_fps = glob.glob("./drive/My Drive/opacity/Nodule154images"+"/"+"*.dcm")
width = []
height = []

for path in tqdm(dicom_fps):
  ds = pydicom.dcmread(path)
  w = ds.Rows
  h = ds.Columns
  width.append(w)
  height.append(h)
  

 95%|█████████▍| 146/154 [03:43<00:10,  1.31s/it]

KeyboardInterrupt: ignored

In [ ]:
data = pd.DataFrame(columns=["Width","Height"])
for x,y in tqdm(zip(width,height)):
  data = data.append({"Width":x,"Height":y},ignore_index=True)


In [ ]:
data

In [ ]:
clndat = pd.concat([train_df,data],axis=1)

In [ ]:
clndat.to_csv("./drive/My Drive/opacity2.csv")

In [ ]:
train_df = pd.read_csv("./drive/My Drive/opacity2.csv")
train_df

In [ ]:
DATA_DIR = '/drive/My Drive/'
ROOT_DIR = "drive/My Drive"

In [ ]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')

!rm -rf .git # to prevent an error when the kernel is committed
!rm -rf images assets # to prevent displaying images at the bottom of a kernel

In [ ]:
sys.path.append(ROOT_DIR+'/Mask_RCNN')

In [ ]:
from mrcnn.config import Config

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
class PneumoniaConfig(Config):
    NAME = "pneumonia"
    NUM_CLASSES = NUM_CATS + 1 # +1 for the background class
    
    GPU_COUNT = 1
    
    BACKBONE = 'resnet50'
    
    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE    
    IMAGE_RESIZE_MODE = 'none'
  
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)

    DETECTION_MIN_CONFIDENCE = 0.85    
    STEPS_PER_EPOCH = 30
    LEARNING_RATE = 0.001
    # STEPS_PER_EPOCH should be the number of instances 
    # divided by (GPU_COUNT*IMAGES_PER_GPU), and so should VALIDATION_STEPS;
    # however, due to the time limit, I set them so that this kernel can be run in 9 hours
    
    
config = PneumoniaConfig()
config.display()

In [ ]:
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

def parse_dataset(dicom_dir, anns): 
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['ImageId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

In [ ]:
train_dicom_dir = 'drive/My Drive/opacity/Nodule154images'
image_fps, image_annotations = parse_dataset(train_dicom_dir, anns=anns)

In [ ]:
# show a few test image detection example
def visualize(): 
    image_id = random.choice(test_image_fps)
    ds = pydicom.read_file(image_id)
    
    # original image 
    image = ds.pixel_array
    
    # assume square image 
    resize_factor = ORIG_SIZE / config.IMAGE_SHAPE[0]
    
    # If grayscale. Convert to RGB for consistency.
    if len(image.shape) != 3 or image.shape[2] != 3:
        image = np.stack((image,) * 3, -1) 
    resized_image, window, scale, padding, crop = utils.resize_image(
        image,
        min_dim=config.IMAGE_MIN_DIM,
        min_scale=config.IMAGE_MIN_SCALE,
        max_dim=config.IMAGE_MAX_DIM,
        mode=config.IMAGE_RESIZE_MODE)

    patient_id = os.path.splitext(os.path.basename(image_id))[0]
    print(patient_id)

    results = model.detect([resized_image])
    r = results[0]
    for bbox in r['rois']: 
        print(bbox)
        x1 = int(bbox[1] * resize_factor)
        y1 = int(bbox[0] * resize_factor)
        x2 = int(bbox[3] * resize_factor)
        y2 = int(bbox[2]  * resize_factor)
        cv2.rectangle(image, (x1,y1), (x2,y2), (77, 255, 9), 3, 1)
        width = x2 - x1 
        height = y2 - y1 
        print("x {} y {} h {} w {}".format(x1, y1, width, height))
    plt.figure() 
    plt.imshow(image, cmap=plt.cm.gist_gray)

visualize()